In [9]:
import os
import csv
import pandas as pd

from collections import Counter

In [10]:
file_list = os.listdir('./Keystrokes/files/')

In [11]:
len(file_list)

168595

In [16]:
key_count_dict = Counter()
for file in file_list:
    print(file)
    df = pd.read_csv('./Keystrokes/files/'+file, sep='\t', encoding='unicode_escape', quoting=csv.QUOTE_NONE)
    key_count_dict.update(df['LETTER'].tolist())

21833_keystrokes.txt
233209_keystrokes.txt
485024_keystrokes.txt
375416_keystrokes.txt
39734_keystrokes.txt
34285_keystrokes.txt
285679_keystrokes.txt
256683_keystrokes.txt
206052_keystrokes.txt
201699_keystrokes.txt
34290_keystrokes.txt
191245_keystrokes.txt
310089_keystrokes.txt
118714_keystrokes.txt
168928_keystrokes.txt
451600_keystrokes.txt
322419_keystrokes.txt
157809_keystrokes.txt
100813_keystrokes.txt
476826_keystrokes.txt
167268_keystrokes.txt
130272_keystrokes.txt
492476_keystrokes.txt
158349_keystrokes.txt
389754_keystrokes.txt
479366_keystrokes.txt
512825_keystrokes.txt
73006_keystrokes.txt
80811_keystrokes.txt
300210_keystrokes.txt
473009_keystrokes.txt
186617_keystrokes.txt
389741_keystrokes.txt
79369_keystrokes.txt
441499_keystrokes.txt
41483_keystrokes.txt
92479_keystrokes.txt
332680_keystrokes.txt
446252_keystrokes.txt
202361_keystrokes.txt
166974_keystrokes.txt
397584_keystrokes.txt
47941_keystrokes.txt
286381_keystrokes.txt
371725_keystrokes.txt
356903_keystrokes.tx

ParserError: Error tokenizing data. C error: Expected 9 fields in line 144, saw 10


In [14]:
df.to_excel('136M_sample.xlsx')